In [3]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.1 MB/s eta 0:00:00


In [4]:
import requests
import json
import pandas as pd
from itertools import combinations
from google.colab import userdata
from faker import Faker
import random
from concurrent.futures import ThreadPoolExecutor
import csv
import sys
from collections import defaultdict
import numpy as np
import os
from faker import Faker

In [5]:
!git clone https://github.com/qwmks/PassportGenerator.git

Cloning into 'PassportGenerator'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 29 (delta 3), reused 29 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 2.02 MiB | 15.42 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [6]:
data_size = 300000
csv_path = "dataset.csv"

"""https://yandex.ru/dev/rasp/doc/ru/reference/schedule-point-point"""
# Определяем константы
base_url="https://api.rasp.yandex.net/v3.0"
yandex_api = userdata.get("yandex_api")

all_station_codes_url = base_url + f"/stations_list/?apikey={yandex_api}&lang=ru_RU&format=json"
all_station_code_json = 'all_station_codes.json'

# Пример данных по классам вагонов
wagon_classes = {
    "1Р": (1, 1),  # Купе-переговорная
    "1В": (1, 50),  # Места 1 класса
    "1С": (1, 45),  # Бизнес-класс
    "2С": (1, 60),  # Эконом-класс
    "2В": (10, 60),  # Экономический+
    "2E": (1, 30),  # Вагон-бистро
    "3Э": (1, 54),  # Плацкарт
    "2Э": (1, 36),  # Купе
    "1Б": (1, 18),  # Бизнес-класс
    "1Л": (1, 18),  # Вагон СВ
    "1А": (1, 4),   # Мягкий вагон с салоном-баром
    "1И": (1, 4)    # Мягкий вагон без бара
}

# Стоимость на километр для каждого класса
class_multipliers = {
    "1Р": 3.5,
    "1В": 3.0,
    "1С": 4.0,
    "2С": 2.5,
    "2В": 2.7,
    "2E": 3.2,
    "3Э": 1.5,
    "2Э": 2.8,
    "1Б": 4.5,
    "1Л": 5.0,
    "1А": 6.0,
    "1И": 5.8
}

kwargs1 = {
    "apikey": yandex_api,
    "format":"json",
    "from":"c146",
    "to":"c213",
    "lang":"ru_RU",
    "page":"1",
    "date":"2024-11-10",
    'transport_types': 'train',  # Фильтрация по типу транспорта
}

# Пример использования для генерации 50,000 записей
stations = ['s9600213', 's9600201', 's9600370']  # Москва, Санкт-Петербург, Казань

# Вероятности банков (префиксы 4 цифры)
bank_distribution = {
    '4276': 0.6,  # Сбербанк
    '4377': 0.2,  # ВТБ
    '4042': 0.1,  # Альфа-Банк
    '5204': 0.05, # Райффайзенбанк
    '5211': 0.05  # Тинькофф
}

# Вероятности платежных систем (префиксы 2 цифры)
system_distribution = {
    '45': 0.5,   # Visa
    '52': 0.3,   # MasterCard
    '22': 0.2    # Мир
}


max_repeats = 5  # Максимум 5 повторений каждой карты
base_price_per_km = 2  # Базовая стоимость за километр

In [14]:
bank_name = {
    'Сбербанк': 0.6,  # Сбербанк
    'ВТБ': 0.2,  # ВТБ
    'Альфа-Банк': 0.1,  # Альфа-Банк
    'Райффайзенбанк': 0.05, # Райффайзенбанк
    'Тинькофф': 0.05  # Тинькофф
}

system_name = {
    'Visa': 0.5,   # Visa
    'MasterCard': 0.3,   # MasterCard
    'Мир': 0.2    # Мир
}

def configure_params(system_distribution, bank_name):
    print('Введите вероятность для банка:')
    for key,value in system_distribution.items():
      system_distribution[key] = input(f"{key}: ")

    for key,value in enumerate(system_distribution):
      bank_distribution[key] = value

    print('Введите вероятность для платежной системы:')
    for key,value in bank_name.items():
      system_distribution[key] = input(f"{key}: ")

    for key,value in enumerate(bank_name):
      system_dis
      tribution[key] = value

# configure_params(system_distribution, bank_name)

In [9]:
# модуль генерации банковских карт

# Проверка валидности номера карты по алгоритму Луна
def luhn_checksum(card_number):
    def digits_of(n):
        return [int(d) for d in str(n)]

    # Убедитесь, что card_number передается как строка
    digits = digits_of(card_number)
    checksum = 0
    odd_digits = digits[-1::-2]
    even_digits = digits[-2::-2]
    checksum += sum(odd_digits)

    for d in even_digits:
        checksum += sum(digits_of(d * 2))

    return checksum % 10

def is_luhn_valid(card_number):
    return luhn_checksum(card_number) == 0

# Генерация валидного номера карты по алгоритму Луна
def generate_luhn_valid_card_number(prefix, length):
    number = [int(x) for x in prefix]

    while len(number) < (length - 1):
        number.append(random.randint(0, 9))

    # Подсчитываем контрольную сумму
    checksum_digit = (10 - luhn_checksum(''.join(map(str, number)))) % 10  # Приведите к строке
    number.append(checksum_digit)

    return ''.join(map(str, number))

# Функция для генерации одной карты
def generate_credit_card(bank_prefixes, bank_probabilities, system_prefixes, system_probabilities, length=16):
    # Выбираем банк на основе вероятностей
    bank_prefix = random.choices(bank_prefixes, bank_probabilities)[0]

    # Выбираем платежную систему на основе вероятностей
    system_prefix = random.choices(system_prefixes, system_probabilities)[0]

    # Полный префикс карты: платежная система + банк
    full_prefix = system_prefix + bank_prefix

    # Генерируем валидный номер карты
    card_number = generate_luhn_valid_card_number(full_prefix, length)

    return card_number

# Генерация набора номеров карт с ограничением повторений
def generate_cards(bank_distribution, system_distribution, num_cards, max_repeats=5):
    bank_prefixes = list(bank_distribution.keys())
    bank_probabilities = list(bank_distribution.values())

    system_prefixes = list(system_distribution.keys())
    system_probabilities = list(system_distribution.values())

    card_count = defaultdict(int)
    cards = []

    while len(cards) < num_cards:
        # Генерация номера карты
        card_number = generate_credit_card(bank_prefixes, bank_probabilities, system_prefixes, system_probabilities)

        # Проверка, сколько раз эта карта уже была сгенерирована
        if card_count[card_number] < max_repeats:
            cards.append(card_number)
            card_count[card_number] += 1

    return cards


In [10]:
# Генерация вагона и места Расчет стоимости билета в зависимости от класса вагона и расстояния

# Функция расчета расстояния по времени в пути
def calculate_distance_from_time(duration_seconds):
    average_speed_kmh = 60  # Средняя скорость поезда
    duration_hours = duration_seconds / 3600
    distance_km = duration_hours * average_speed_kmh
    return round(distance_km, 2)

# Генерация вагона и места
def generate_wagon_seat(duration_seconds):
    # Рассчитываем расстояние
    distance_km = calculate_distance_from_time(duration_seconds)

    # Случайно выбираем класс вагона
    wagon_class = random.choice(list(wagon_classes.keys()))
    num_wagons, seats_per_wagon = wagon_classes[wagon_class]

    # Генерируем случайный вагон и место
    wagon_number = random.randint(1, num_wagons)
    seat_number = random.randint(1, seats_per_wagon)

    # Возвращаем информацию о вагоне и месте
    return f"{wagon_number}-{seat_number}"

# Расчет стоимости билета
def ticket_price(duration_seconds):
    # Рассчитываем расстояние
    distance_km = calculate_distance_from_time(duration_seconds)

    # Случайно выбираем класс вагона
    wagon_class = random.choice(list(wagon_classes.keys()))

    # Рассчитываем стоимость билета
    ticket_price = distance_km * base_price_per_km * class_multipliers[wagon_class]

    return f"{round(ticket_price, 2)} руб"

In [11]:
# Модуль получения всех доступных станций

""" Docs: https://yandex.ru/dev/rasp/doc/ru/reference/stations-list """
def get_all_station_codes(all_station_codes_url = all_station_codes_url, all_station_code_json = all_station_code_json):
  response = requests.get(url=all_station_codes_url)
  response.raise_for_status()
  data = response.json()  # Преобразование ответа в формат JSON

  # Сохранение данных в JSON файл
  with open(all_station_code_json, 'w', encoding='utf-8') as json_file:
      json.dump(data, json_file, ensure_ascii=False, indent=4)  # Запись в файл с отступами для удобства чтения

  print(f"Response saved to {all_station_code_json}")

In [12]:
""" Docs: https://yandex.ru/dev/rasp/doc/ru/reference/schedule-point-point """
"""https://github.com/FasterXaos/Algorithms_and_data_structures/tree/master/Dataset_generation"""
def timetable_between_stations(params, timetable_between_stations_url = (base_url + "/search/")):
    # Переменная для хранения всех результатов
    all_trains = []


    response = requests.get(timetable_between_stations_url, params=params)
    response.raise_for_status() # тут проверяем успешен ли запрос без ==200

    data = response.json()
    print(data)

    search = data.get("search")

    from_station = search.get("from").get("title")
    to_station = search.get("to").get("title")


    # Парсим данные о каждом сегменте
    for segment in data.get("segments"):
        thread = segment.get('thread', {})
        tickets_info = (segment.get('tickets_info', {}) or {}).get('places', [{}])

        if(tickets_info):
          curensy = tickets_info[0].get('currency')
          price = (tickets_info[0].get('price', {}) or {}).get('whole', {})
        else:
          curensy = False
          price = False

        duration = segment.get("duration", 3600)

        # Добавляем информацию по поезду
        all_trains.append({

            'train_number': thread.get('number', 'N/A'),
            'from_station': from_station,
            'to_station': to_station,
            'wagon_seat': generate_wagon_seat(duration),
            'price': str(price + curensy) if price else ticket_price(duration),
            'departure': segment.get('departure', 'N/A'),
            'arrival': segment.get('arrival', 'N/A'),

            # 'train_type': thread.get('transport_subtype', {}).get('title', 'N/A'),
        })


    # Возвращаем результаты в виде DataFrame
    return pd.DataFrame(all_trains)

# timetable_between_stations(kwargs1)

In [16]:
def combine_dataframes_n_to_1(df1, df2, n):
    """
    Объединяет два DataFrame таким образом, чтобы каждая n-я строка df1 сопоставлялась строкам df2.

    Параметры:
    df1 (pd.DataFrame): Первый DataFrame с n строками.
    df2 (pd.DataFrame): Второй DataFrame, который будет повторен для каждой группы из n строк df1.
    n (int): Число строк из df1, которым будет сопоставляться одна строка из df2.

    Возвращает:
    pd.DataFrame: Новый DataFrame, где строки df2 сопоставлены строкам df1.
    """

    # Проверяем, что df2 не пуст
    if df2.empty:
        raise ValueError("Второй DataFrame df2 не должен быть пустым.")

    # Количество строк в df1
    len_df1 = len(df1)

    # Рассчитываем, сколько строк нужно добавить из df2
    rows_needed = int(np.round((len_df1 + n - 1) // n * n))  # Округляем вверх до ближайшего кратного n

    # Повторяем строки df2 до необходимого размера
    repeat_count =  np.round(rows_needed // len(df2)) + 1  # Количество повторений для df2
    df2_repeated = pd.concat([df2] * int(repeat_count), ignore_index=True)

    # Обрезаем df2_repeated до нужного размера
    df2_repeated = df2_repeated.head(rows_needed)

    # Сопоставляем два DataFrame по индексам
    combined_df = pd.concat([df1.reset_index(drop=True), df2_repeated.reset_index(drop=True)], axis=1)

    return combined_df

# Инициализация Faker для русского языка
fake = Faker('ru_RU')

# Количество записей
num_records = 10

# Серия и номер паспорта
def passport_number():
    nums = [
        random.randint(0, 9)
        for _ in range(0, 10)
    ]

    return ''.join(map(str, nums[:4])) + ' ' + ''.join(map(str, nums[4:]))

def generate_dataset(data_size):
    train_data = timetable_between_stations(kwargs1)


    # os.chdir(os.path.dirname("/content/PassportGenerator/data"))
    # sys.path.append('/content/PassportGenerator')
    # sys.path.append('/content/PassportGenerator/data')
    # from genPassportData import generateData
    # other_data = generateData(data_size)
    # os.chdir(os.path.dirname("/content/"))

    # other_data = pd.DataFrame(other_data)

    fake_data = pd.DataFrame({
        "full_name": [fake.name() for _ in range(data_size)],
        "pasport_number": [passport_number() for _ in range(data_size)],
        "card_number": generate_cards(bank_distribution, system_distribution, num_cards=data_size)
    })

    # Для каждых data_size / len(train_data.columns) строк из df1 сопоставляем одну строку из df2
    combined_df = combine_dataframes_n_to_1(fake_data, train_data, int(data_size / len(train_data.columns)))

    return combined_df

generate_dataset(data_size).to_csv(csv_path)
print(f"Сгенерировано {data_size} билетов и сохранено в {csv_path} .")

# timetable_between_stations(kwargs1)

{'search': {'from': {'type': 'settlement', 'title': 'Симферополь', 'short_title': 'Симферополь', 'popular_title': 'Симферополь', 'code': 'c146'}, 'to': {'type': 'settlement', 'title': 'Москва', 'short_title': 'Москва', 'popular_title': 'Москва', 'code': 'c213'}, 'date': '2024-11-10'}, 'segments': [{'thread': {'number': '028С', 'title': 'Симферополь — Москва', 'short_title': 'Симферополь — Москва', 'express_type': None, 'transport_type': 'train', 'carrier': {'code': 63438, 'title': 'Гранд Сервис Экспресс (Таврия)', 'codes': {'sirena': None, 'iata': None, 'icao': None}, 'address': None, 'url': 'http://grandtrain.ru', 'email': None, 'contacts': '', 'phone': '', 'logo': None, 'logo_svg': None}, 'uid': '028S_23_2', 'vehicle': None, 'transport_subtype': {'title': None, 'code': None, 'color': None}, 'thread_method_link': 'api.rasp.yandex.net/v3/thread/?date=2024-11-10&uid=028S_23_2'}, 'stops': '', 'from': {'type': 'station', 'title': 'Симферополь-Пасс.', 'short_title': '', 'popular_title': ''

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
_df_15.groupby('train_number').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['wagon_seat'].value_counts()
    for x_label, grp in _df_19.groupby('train_number')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('train_number')
_ = plt.ylabel('wagon_seat')